In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import product
import statsmodels.api as sm

import tensorflow as tf

## These 3 lines are from the original notebook.
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.layers import LSTM

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Activation, Dense, Dropout
from keras.layers import LSTM

from itertools import cycle
import plotly.offline as py
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

plt.style.use('seaborn-darkgrid')

# Transformation of time series into supervision problems

Time series data needs to be prepared before training a supervised learning model (such as LSTM neural network). For example, a unary time series is represented as an observation vector:

```
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

```

The supervised learning algorithm requires data to be provided as a set of samples, where each sample has an input component X and an output component y. The model will learn how to map input to output from the provided examples.


The time series must be converted into samples with input and output components .  For a univariate time series problem that is interested in one-step forecasting, you can use the observations at the previous time point as input and the observations at the current time point as the output. For example, the above 10-step univariate sequence can be expressed as a supervised learning problem, the input is 3 time steps, and the output is 1 time step, as shown below:

X *******   Y

===============
 
[1, 2, 3],  [4]

[2, 3, 4],  [5]

[3, 4, 5],  [6]

In [ ]:
# Google Drive Root Path
# root_path = '/content/drive/MyDrive/Datasets_For_Working/Bitcoin_2015-2021-30-Oct-2021/3_pull.csv'

# Local Machine Root Path
# root_path = './input/btc_ohlc_Oct_2015_to_Oct_2021.csv'

# Kaggle Root Path
root_path = 'btc_ohlc_Oct_2015_to_Oct_2021.csv'


# btc_input_df = pd.read_csv(root_path, nrows=500)
btc_input_df = pd.read_csv(root_path)
# btc_input_df = (btc_input_df.set_index('close')
#                 .applymap('{:,.2f}'.format))
# btc_input_df = btc_input_df.rename_axis('time').reset_index()

btc_input_df.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'btc_ohlc_Oct_2015_to_Oct_2021.csv'

In [ ]:
btc_input_df.shape

In [ ]:
btc_no_time_df = btc_input_df.drop(columns=['time'])
btc_no_time_df

In [ ]:
btc_input_df.describe()

In [ ]:
btc_input_df.info()

# Convert 'time' column from object dtype to datetime dtype

From above I can see that the 'time' column is being treated as an object rather than as dates. To fix this, I will use the (pd.to_datetime() function which converts the arguments to dates.

In [ ]:
btc_input_df_datetype = btc_input_df.astype({'time': 'datetime64'})

In [ ]:
btc_input_df_datetype.info()

In [ ]:
print('Null Values:',btc_input_df_datetype.isnull().values.sum())
print('If any NA values:', btc_input_df_datetype.isnull().values.any())

In [ ]:
btc_input_df_datetype.tail()

In [ ]:
btc_input_df_datetype.head()

In [ ]:
btc_input_df_datetype.set_index("time").close.plot(figsize=(24,7), title="Bitcoin Weighted Price")

# Lag Plot

Lag plot are used to observe the autocorrelation. These are crucial when we try to correct the trend and stationarity and we have to use smoothing functions. Lag plot helps us to understand the data better.

In [ ]:
plt.figure(figsize=(15,12))
plt.suptitle('Lag Plots', fontsize=22)

plt.subplot(3,3,1)
pd.plotting.lag_plot(btc_input_df_datetype['close'], lag=1) #minute lag
plt.title('1-Minute Lag')

plt.subplot(3,3,2)
pd.plotting.lag_plot(btc_input_df_datetype['close'], lag=60) #hourley lag
plt.title('1-Hour Lag')

plt.subplot(3,3,3)
pd.plotting.lag_plot(btc_input_df_datetype['close'], lag=1440) #Daily lag
plt.title('Daily Lag')

plt.subplot(3,3,4)
pd.plotting.lag_plot(btc_input_df_datetype['close'], lag=10080) #weekly lag
plt.title('Weekly Lag')

plt.subplot(3,3,5)
pd.plotting.lag_plot(btc_input_df_datetype['close'], lag=43200) #month lag
plt.title('1-Month Lag')

plt.legend()
plt.show()

We can see that there is a positive correlation for minute, hour and daily lag plots. 

Correlation decreases drastically with Weekly Lag and absolutely no correlation for month lag plots.

It makes sense to re-sample our data atmost at the Daily level, thereby preserving the autocorrelation as well.

# Making data ready for LSTM - pd.to_datetime(unit='s) and then groupby('date')

## pd.to_datetime unit='s'

The unit of the arg (D,s,ms,us,ns) denote the unit, which is an integer or float number. This will be based off the origin. Example, with unit=’ms’ and origin=’unix’ (the default), this would calculate the number of milliseconds to the unix epoch start.

Say you pass an int as your arg (like 20203939), with unit, you’ll be able specify what unit your int is away from the origin. In the example here, if we set unit=’s’, this means pandas will interpret 20203939 as 20,203,939 seconds away from the origin. Available units are [D,s,ms,us,ns]. 
[Source](https://www.dataindependent.com/pandas/pandas-to-datetime/)

In [ ]:
btc_input_df_datetype['date'] = pd.to_datetime(btc_input_df_datetype['time'],unit='s').dt.date

# display(btc_input_df_datetype.tail())

group = btc_input_df_datetype.groupby('date')
btc_closing_price_groupby_date = group['close'].mean()
btc_closing_price_groupby_date

In [ ]:
display(btc_closing_price_groupby_date.head(10))

print("Length of btc_closing_price_groupby_date :", len(btc_closing_price_groupby_date))

# Train Test Split

In [ ]:
prediction_days = 60

# Set Train data to be uplo ( Total data length - prediction_days )
df_train= btc_closing_price_groupby_date[:len(btc_closing_price_groupby_date)-prediction_days].values.reshape(-1,1)


# Set Test data to be the last prediction_days (or 60 days in this case)
df_test= btc_closing_price_groupby_date[len(btc_closing_price_groupby_date)-prediction_days:].values.reshape(-1,1)
# df_train
df_test

In [ ]:
df_test.shape


In [ ]:
chosen_col = 'Close'

fig, ax = plt.subplots(1, figsize=(13, 7))

ax.plot(df_train, label='Train', linewidth=2)
ax.plot(df_test, label='Test', linewidth=2)
ax.set_ylabel('Price USD', fontsize=14)
ax.set_title('BitCoin Closeprice Train-Test Data', fontsize=16)
ax.legend(loc='best', fontsize=16)

# Mix Max Scaling of Data post Train-Test Split

Scaling must be done after the data has been split into training and test sets — with each being scaled separately. 

A common mistake when first using the LSTM is to first normalize the data before splitting the data.

The reason this is erroneous is that the normalization technique will use data from the test sets as a reference point when scaling the data as a whole. This will inadvertently influence the values of the training data, essentially resulting in data leakage from the test sets.


In [ ]:
scaler_train = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler_train.fit_transform(df_train)

scaler_test = MinMaxScaler(feature_range=(0, 1))
scaled_test = scaler_test.fit_transform(df_test)



# Dataset Generator for LSTM

We will frame the problem to take a window of the last so many number of days of data to predict the current days data.

To achieve this, we will define a new function named `dataset_generator_lstm()` that will split the input sequence into windows of data appropriate for fitting a supervised learning model, like an LSTM


For example, if the sequence was:

1, 2, 3, 4, 5, 6, 7, 8, 9, 10


Then the samples for training the model will look like:

```

Input 				| Output
1, 2, 3, 4, 5 		6
2, 3, 4, 5, 6 		7
3, 4, 5, 6, 7 		8

```

LSTMs expect each sample in the dataset to have two dimensions; the first is the number of time steps (in the above case it is 5), and the second is the number of observations per time step (in this case it is 1).

Because it is a regression type problem, we will use a linear activation function in the output layer and optimize the mean squared error loss function. We will also evaluate the model using the mean squared error (MAE) metric.

## Define look_back period

A “lookback period” defines how many previous timesteps are used in order to predict the subsequent timestep. 

For example if I set the lookback period is to 5, that that means that I am using the time steps at t-4, t-3, t-2, t-1, and t to predict the value at time t+1.

For my case below, I will be using a one-step prediction model.

### Lookback period


```py

lookback = 5

X_train, Y_train = dataset_generator_lstm(train, lookback)

X_val, Y_val = dataset_generator_lstm(val, lookback)

```

### Quick note on Python slice notation for slicing an array

- [1:5] is equivalent to "from 1 to 5" (5 not included)

- [1:] is equivalent to "1 to end"

- [len(a):] is equivalent to "from length of a to end"

In the below function I will generate the train_X array and train_y array for feeding into the LSTM

In [ ]:
def dataset_generator_lstm(dataset, look_back=5):
    # A “lookback period” defines the window-size of how many
    # previous timesteps are used in order to predict
    # the subsequent timestep. 
    dataX, dataY = [], []
    
    for i in range(len(dataset) - look_back):
        window_size_x = dataset[i:(i + look_back), 0]
        dataX.append(window_size_x)
        dataY.append(dataset[i + look_back, 0]) # this is the label or actual y-value
    return np.array(dataX), np.array(dataY)

trainX, trainY = dataset_generator_lstm(scaled_train)

testX, testY = dataset_generator_lstm(scaled_test)

print("trainX: ", trainX.shape)
print("trainY: ", trainY.shape)
print("testY: ", testX.shape)
print("testY", testY.shape)
## TTTTTTTTTTTTTTT

In [ ]:
print("trainX: ", trainX)
# print("trainY: ", trainY)
# print("testY: ", testX)
# print("testY", testY)

# For LSTM I need to reshape input to be a 3D Tensor of [samples, time steps, features]

    X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

**Samples** - This is the len(dataX), or the amount of data points you have.

**Time steps** - A sample contains multiple time steps, that is, the width of the sliding window (according to the above example, the time step is 3). Note here that it is distinguished from the sliding step of the sliding window. This is equivalent to the amount of time steps you run your recurrent neural network. If you want your network to have memory of 60 characters, this number should be 60. For this notebook, I am using the window-size to be 5.

**Features** - this is the amount of features in every time step. If you are processing pictures, this is the amount of pixels. In this case I have 1 feature (the price of Bitcoin) per time step.

 ### According to the documentation and the source code, the Keras LSTM input data must be in the form: [batch_size, timesteps, input_dim].

 In Keras, the number of time steps is equal to the number of LSTM cells. This is what the word “time steps” means in the 3D tensor of the shape [batch_size, timesteps, input_dim].


### So to emphasize again, the input to every LSTM layer must be three-dimensional.

The three dimensions of this input are:

Samples. One sequence is one sample. A batch is comprised of one or more samples.

Time Steps. One time step is one point of observation in the sample.

Features. One feature is one observation at a time step.

This means that the input layer expects a 3D array of data when fitting the model and when making predictions, even if specific dimensions of the array contain a single value, e.g. one sample or one feature.

When defining the input layer of your LSTM network, the network assumes you have 1 or more samples and requires that you specify the number of time steps and the number of features. You can do this by specifying a tuple to the “input_shape” argument.

-----------------------

### Example of LSTM With Single Input Sample where you have one sequence of multiple time steps and one feature.

For example, this could be a sequence of 10 values:

```
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0

```

1. We can define this sequence of numbers as a NumPy array.

2. We can then use the reshape() function on the NumPy array to reshape this one-dimensional array into a three-dimensional array with 1 sample, 10 time steps, and 1 feature at each time step.

The reshape() function when called on an array takes one argument which is a tuple defining the new shape of the array. We cannot pass in any tuple of numbers; the reshape must evenly reorganize the data in the array.

3. Once reshaped, we can print the new shape of the array.

Putting all of this together, the complete example is listed below.

```py
from numpy import array
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
data = data.reshape((1, 10, 1))
print(data.shape)

```
Running the example prints the new 3D shape of the single sample.

```
(1, 10, 1)

```
This data is now ready to be used as input (X) to the LSTM with an input_shape of (10, 1).

```py
model = Sequential()
model.add(LSTM(32, input_shape=(10, 1)))
model.add(Dense(1))

```

----------------------------------

-------------------------------

# For LSTM Reshape input ( trainX and testX ) to be 3-D of [samples, time steps, features]

### First check the current shape of trainX and testX

In [ ]:
print(trainX.shape)
print(testX.shape)

### And now reshape trainX and testX 

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1 ))

print("Shape of trainX: ", trainX.shape)
print("Shape of testX: ", testX.shape)

In [ ]:
print("trainX: ", trainX)
print(" ********** ")
print("testX: ", testX)

## Let’s take a look at the normalized window for trainX.

```
[[[0.00000000e+00]
  [1.44159882e-04]
  [2.79473948e-04]
  [5.34851927e-04]
  [4.90695552e-04]]

 [[1.44159882e-04]
  [2.79473948e-04]
  [5.34851927e-04]
  [4.90695552e-04]
  [4.21819458e-04]]

 [[2.79473948e-04]
  [5.34851927e-04]
  [4.90695552e-04]
  [4.21819458e-04]
  [7.33745838e-04]]... ]

```
Above are the first three entries. We can see that the five time steps immediately prior to the one we are trying to predict move in a stepwise motion.

Consider the last value of the first window which is => 4.90695552e-04

# LSTM (Long Short-Term Memory) Mechanism

The Long Short-Term Memory, or LSTM, is a recurrent neural network that is comprised of internal gates.

Unlike other recurrent neural networks, the network’s internal gates allow the model to be trained successfully using backpropagation through time, or BPTT, and avoid the vanishing gradients problem.

In the Keras deep learning library, LSTM layers can be created using the LSTM() class.

Creating a layer of LSTM memory units allows you to specify the number of memory units within the layer.

Each unit or cell within the layer has an internal cell state, often abbreviated as “c“, and outputs a hidden state, often abbreviated as “h“.

The Keras API allows you to access these data, which can be useful or even required when developing sophisticated recurrent neural network architectures, such as the encoder-decoder model.


## `input_shape` of LSTM Model - `input_shape` is supposed to be (timesteps, n_features).

    input_shape = (95000,360)

Keras LSTM takes and input with shape of (n_examples, n_times, n_features) and your layers input has to have this shape

To give you an example: if I'm observing the amount of rain and the temperature each hour for 24h in order to predict the weather (1 = good, 0 = bad), and I do that for 365 days, I will have 365 samples, each of which will have 24 timesteps, and 2 variables (one for rain, one for temperature), so my input is going to have the shape (365, 24, 2), and input_shape = (24, 2)


## `return_sequences=True`

The original LSTM model is comprised of a single hidden LSTM layer followed by a standard feedforward output layer.

The Stacked LSTM is an extension to this model that has multiple hidden LSTM layers where each layer contains multiple memory cells.

You must set return_sequences=True when stacking LSTM layers so that the second LSTM layer has a compatible n-dimensional sequence input. 

#### To stack LSTM layers, we need to change the configuration of the prior LSTM layer to output a compatible n-dim array as input for the subsequent layer.

We can do this by setting the return_sequences argument on the layer to True (defaults to False). This will return one output for each input time step and provide the compatible n-dim array.

## Why Increase Depth of an LSTM ?

Stacking LSTM hidden layers makes the model deeper, more accurately earning the description as a deep learning technique.

It is the depth of neural networks that is generally attributed to the success of the approach on a wide range of challenging prediction problems.

Additional hidden layers can be added to a Multilayer Perceptron neural network to make it deeper. The additional hidden layers are understood to recombine the learned representation from prior layers and create new representations at high levels of abstraction. For example, from lines to shapes to objects.

A sufficiently large single hidden layer Multilayer Perceptron can be used to approximate most functions. Increasing the depth of the network provides an alternate solution that requires fewer neurons and trains faster. Ultimately, adding depth it is a type of representational optimization.

Given that LSTMs operate on sequence data, it means that the addition of layers adds levels of abstraction of input observations over time. In effect, chunking observations over time or representing the problem at different time scales.

Stacked LSTMs are now a stable technique for challenging sequence prediction problems. 

# Final LSTM 

In [ ]:
# First checking the values for input_shape = (trainX.shape[1], trainX.shape[2])
# Note - `input_shape` of LSTM Model - `input_shape` is supposed to be (timesteps, n_features).

print("trainX.shape[1] - i.e. timesteps in input_shape = (timesteps, n_features) ", trainX.shape[1])
print("trainX.shape[2] - i.e. n_features in input_shape = (timesteps, n_features) ", trainX.shape[2])

In [ ]:
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
# You must set return_sequences=True when stacking LSTM layers so that the second LSTM layer
# has a compatible n-dimensional sequence input.
# This hyper parameter should be set to False (which is the default value) for the last layer
# and true for the other previous layers.

regressor.add(LSTM(units = 128, activation = 'relu',return_sequences=True, input_shape = (trainX.shape[1], trainX.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 64, input_shape = (trainX.shape[1], trainX.shape[2])))
# regressor.add(LSTM(units = 64, return_sequences = True, input_shape = (trainX.shape[1], trainX.shape[2])))
regressor.add(Dropout(0.2))
# Note - If I plan to add 3-rd or 4-th layers of LSTM then 
# I must set return_sequences=True in the 2-nd layer above
# so that the 3-rd LSTM layer has a compatible n-dimensional sequence input.


# Adding a third LSTM layer and some Dropout regularisation
# regressor.add(LSTM(units = 64, return_sequences = True, input_shape = (trainX.shape[1], trainX.shape[2])))
# regressor.add(Dropout(0.2))


# Adding a fourth LSTM layer and some Dropout regularisation
# regressor.add(LSTM(units = 64, input_shape = (trainX.shape[1], trainX.shape[2])))
# regressor.add(Dropout(0.2))


# Adding the output layer
regressor.add(Dense(units = 1))

regressor.summary()


In [ ]:
from  keras.callbacks import ModelCheckpoint, EarlyStopping

# Compiling the LSTM
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

checkpoint_path = 'my_best_model.hdf5'

checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')


earlystopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

callbacks = [checkpoint, earlystopping]


history = regressor.fit(trainX, trainY, batch_size = 32, epochs = 300, verbose=1, shuffle=False, validation_data=(testX, testY), callbacks=callbacks)

## Plot line graph to show Loss Numbers relative to the epoch

In [ ]:
from  keras.models import load_model

model_from_saved_checkpoint = load_model(checkpoint_path)

In [ ]:
plt.figure(figsize=(16,7))
plt.plot(history.history['loss'], label='train')

plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

# LSTM Predictions using testX and plotting line graph against Actual testY

Due to scaling step done earlier with MinMaxScaler the predicted scale is between 0 and 1. 
Now, I need to transfer this scale to the original data scale (real value). for example:[0.58439621 0.58439621 0.58439621 ... 0.81262134 0.81262134 0.81262134], the pred answer transfer to :[250 100 50 60 .....]
So here I am going to use `inverse_transform` to Scale back the data to the original representation.

In [ ]:
# Transformation to original form and making the predictions

# predicted_btc_price_test_data = regressor.predict(testX)
predicted_btc_price_test_data = model_from_saved_checkpoint.predict(testX)

predicted_btc_price_test_data = scaler_test.inverse_transform(predicted_btc_price_test_data.reshape(-1, 1))

test_actual = scaler_test.inverse_transform(testY.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(16,7))

plt.plot(predicted_btc_price_test_data, 'r', marker='.', label='Predicted Test')

plt.plot(test_actual, marker='.', label='Actual Test')

plt.legend()
plt.show()

# LSTM Prediction using trainX and plotting line graph against Actual trainY

In [ ]:
# Transformation to original form and making the predictions

predicted_btc_price_train_data = model_from_saved_checkpoint.predict(trainX)

predicted_btc_price_train_data = scaler_train.inverse_transform(predicted_btc_price_train_data.reshape(-1, 1))

train_actual = scaler_train.inverse_transform(trainY.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(16,7))

plt.plot(predicted_btc_price_train_data, 'r', marker='.', label='Predicted Train')

plt.plot(train_actual, marker='.', label='Actual Train')

plt.legend()
plt.show()

# RMSE - Test Data

In [ ]:
rmse_lstm_test = math.sqrt(mean_squared_error(test_actual, predicted_btc_price_test_data))

print('Test RMSE: %.3f' % rmse_lstm_test)

# With 2 Layers + Drop
# out + lookback=5 => I got - Test RMSE: 1666.162  => This seems best


# RMSE - Train Data

In [ ]:
rmse_lstm_train = math.sqrt(mean_squared_error(train_actual, predicted_btc_price_train_data))
print('Test RMSE: %.3f' % rmse_lstm_train)

# With 2 Layers + Dropout + lookback=5 => I got - Test RMSE: 1047.916  => This seems best